In [1]:
import shapefile
import argparse
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import os
import rasterio

In [2]:
def find_middle(points):
    max_lat = points[0][1] ## north
    max_long = points[0][0] ## east
    min_lat = points[0][1] ## south
    min_long = points[0][0] ## west
    for vert in points:
        if vert[0] < min_long:
            min_long = vert[0]
        if vert[0] > max_long:
            max_long = vert[0]
        if vert[1] < min_lat:
            min_lat = vert[1]
        if vert[1] > max_lat:
            max_lat = vert[1]
    ave_lat = (max_lat + min_lat) / 2
    ave_long = (max_long + min_long) / 2
    return (ave_long, ave_lat)

def find_nearest_ground_elev(pband1, row, col):
    cr_dist = 501
    c_right = -9999.0
    for cr in range(500):
        try:
            if pband1[row,col+cr] != -9999.0:
                c_right = pband1[row,col+cr]
                cr_dist = cr
                break
        except IndexError:
            break
    cd_dist = 501
    c_down = -9999.0
    for cd in range(500):
        try:
            if pband1[row+cd,col] != -9999.0:
                c_down = pband1[row+cd,col]
                cd_dist = cd
                break
        except IndexError:
            break
    cl_dist = 501
    c_left = -9999.0
    for cl in range(500):
        if col-cl < 0:
            break
        elif pband1[row,col-cl] != -9999.0:
            c_left = pband1[row,col-cl]
            cl_dist = cl
            break
    cu_dist = 501
    c_up = -9999.0
    for cu in range(500):
        if row-cu < 0:
            break
        elif band1[row-cu,col] != -9999.0:
            c_up = pband1[row-cu,col]
            cu_dist = cu
            break
    print(c_right, cr_dist, c_down, cd_dist, c_left, cl_dist, c_up, cu_dist)
    heights = [c_right, c_down, c_left, c_up]
    dists = [cr_dist, cd_dist, cl_dist, cu_dist]
    all_9999 = True
    if max(heights) == -9999.0:
        print("ERROR: STOP NOW AND INSPECT")
        print("ERROR: there's no terrain within 500m of the building to base the ground_elev on")
    nearest_grnd = heights[dists.index(min(dists))]
    return nearest_grnd

In [3]:
lat_deg = 111320
long_deg = 40075000 * np.cos(lat_deg) / 360

In [4]:
print(lat_deg, long_deg)

111320 77083.48825393952


In [5]:
def measure(lat1, lon1, lat2, lon2):  ## generally used geo measurement function
    R = 6378.137 ## Radius of earth in KM
    dLat = lat2 * np.pi / 180 - lat1 * np.pi / 180
    dLon = lon2 * np.pi / 180 - lon1 * np.pi / 180
    a = np.sin(dLat/2) * np.sin(dLat/2) + \
        np.cos(lat1 * np.pi / 180) * np.cos(lat2 * np.pi / 180) * \
        np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c;
    return d * 1000 # meters

In [6]:
lat_deg = measure(40, -105, 39, -105)
long_deg = measure(40.0096, -105, 40.0096, -106)

In [7]:
print(lat_deg, long_deg)

111319.49079327371 85263.23969113332


In [8]:
lat_deg = 110950
long_deg = 85000

In [9]:
path = "/home/jovyan/shared/max/rf_rt/helper_files/boulder_campus/holey_Bs/boulder_bs/"

In [10]:
# select_buildings = [971604]
# select_buildings = [972192] ## library
select_buildings = [969073]

In [11]:
shapefile_f = "Buildings/Building_Footprints.shp"

In [12]:
sf = shapefile.Reader(shapefile_f)

In [13]:
shapes = sf.shapes()
rec = sf.records()

In [14]:
tmp_vertex_degrees = []
a_building_height = []
a_building_low = []
a_building_id = []
a_holey_building = []
## chosen lower left base my 0m, 0m orientation on
common_0_long_lat = (-105.27368, 40.00707)
## epsg format (bottom, left)
common_0_epsg32613 =  (476641.151340244, 4428577.785411156)
terrain_dataset = rasterio.open("terrain/hellems_arap_1m.tin.tif")
band1 = terrain_dataset.read(1)

In [15]:
for i in range(len(shapes)):
    if rec[i]['DRCOGID'] in select_buildings:
        print("Building is {}".format(rec[i]['DRCOGID']))
        print(shapes[i].points)
        middle_point_of_b = find_middle(shapes[i].points)
        print(middle_point_of_b)
        z_meters = (middle_point_of_b[0] - common_0_long_lat[0]) * long_deg
        x_meters = (middle_point_of_b[1] - common_0_long_lat[1]) * lat_deg
        print(x_meters, z_meters)
        print(x_meters + common_0_epsg32613[1], z_meters + common_0_epsg32613[0])
        row, col = terrain_dataset.index(z_meters + common_0_epsg32613[0], x_meters + common_0_epsg32613[1])
        ground_elev = band1[row,col] ## if this fails it probably means you have buildings that are outside of the terrain
        print(ground_elev)
        if ground_elev == -9999.0:
            ground_elev = find_nearest_ground_elev(band1, row, col)
        print("new ground elev = ", ground_elev)          
        print("-----------------------")
        print(shapes[i].parts)
        tmp = []
        for vert in shapes[i].points:
            tmp.append(np.array([copy(vert[0]),copy(vert[1])]))
        if len(shapes[i].parts) > 1: ## marks building with holes
            a_holey_building.append(shapes[i].parts)
        else:
            a_holey_building.append(False)
        tmp_vertex_degrees.append(tmp)
        a_building_height.append(rec[i]['BLDGHEIGHT']*0.3048+ground_elev)
        a_building_low.append(ground_elev-20) ## send walls below the ground
        a_building_id.append(rec[i]['DRCOGID'])
        # if (len(a_building_id)> 10500):
        #     break

In [13]:
a_building_height

[1711.7099609375, 1732.5799560546875, 1664.4363609375, 1674.1831560546875]

In [11]:
a_vertex_degrees = np.array(tmp_vertex_degrees, dtype=object)

In [12]:
print("num buildings to write: {}".format(len(a_vertex_degrees)))
print("total number of objects: {}".format(a_vertex_degrees.size))

num buildings to write: 1
total number of objects: 46


In [20]:
## longitude, latitude
lower_left = np.array([-105.27429, 40.00643])
upper_right = np.array([-105.27377, 40.00832])
upper_left = np.array([-105.27512, 40.00839])

In [21]:
vc = 0
bc = 0

xx = []
yy = []

In [22]:
for i in range(len(a_vertex_degrees)):
    xx = []
    yy = []
    print("building height is ", a_building_height[i])
    building_high = a_building_height[i] * 0.3048 ## convert from feet to meters
    building_low = a_building_low[i] * 0.3048 ## convert from feet to meters
    v_list = []
    f_list = []
    building_vc = 0
    meter_v = []
    for j in range(0, len(a_vertex_degrees[i])):
        xx.append((a_vertex_degrees[i][j][0] - upper_left[0]) * long_deg)
        yy.append((upper_left[1] - a_vertex_degrees[i][j][1]) * lat_deg)
        if j == 0:
            meter_v.append((a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg]))
            vertex = (a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg])
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_low, vertex[0]))
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_high, vertex[0]))
            vc += 2
            building_vc += 2
        elif j == len(a_vertex_degrees[i])-1:
            f_list.append("f {} {} {}".format(vc-1, vc-building_vc+1, vc-building_vc+2))
            f_list.append("f {} {} {}".format(vc-1, vc-building_vc+2, vc))
            # if a_holey_building[i]:
            #     meter_v.append((a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg]))
        else:
            meter_v.append((a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg]))
            vertex = (a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg])
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_low, vertex[0]))
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_high, vertex[0]))
            vc += 2
            building_vc += 2
            f_list.append("f {} {} {}".format(vc-3, vc-1, vc))
            f_list.append("f {} {} {}".format(vc-3, vc, vc-2))

building height is  5441


In [23]:
v_list

['v 161.2415 1655.9784 -7.3797',
 'v 161.2415 1658.4168 -7.3797',
 'v 162.4992 1655.9784 -7.3557',
 'v 162.4992 1658.4168 -7.3557',
 'v 162.3963 1655.9784 -2.0183',
 'v 162.3963 1658.4168 -2.0183',
 'v 152.1397 1655.9784 -2.2144',
 'v 152.1397 1658.4168 -2.2144',
 'v 152.2424 1655.9784 -7.5518',
 'v 152.2424 1658.4168 -7.5518',
 'v 156.9394 1655.9784 -7.4620',
 'v 156.9394 1658.4168 -7.4620']

In [24]:
f_list

['f 1 3 4',
 'f 1 4 2',
 'f 3 5 6',
 'f 3 6 4',
 'f 5 7 8',
 'f 5 8 6',
 'f 7 9 10',
 'f 7 10 8',
 'f 9 11 12',
 'f 9 12 10',
 'f 11 1 2',
 'f 11 2 12']

In [25]:
meter_v

[array([-7.379748938376451, 161.24152802331935], dtype=object),
 array([-7.355654670746503, 162.49915255604606], dtype=object),
 array([-2.0183293781173006, 162.39628429098474], dtype=object),
 array([-2.214373726516783, 152.13973864961935], dtype=object),
 array([-7.551791844377931, 152.2424071711594], dtype=object),
 array([-7.462034565648423, 156.93936461934152], dtype=object)]

In [19]:
try:
    band1[1,5000]
except IndexError:
    print("ope!")

ope!
